<a href="https://colab.research.google.com/github/anujsahani01/EngMarBART/blob/main/Dataset_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install --upgrade accelerate
!pip install wandb
!pip install --no-cache-dir transformers sentencepiece
!pip install evaluate
!pip install rouge_score
!pip install sacremoses
!pip install fasttext
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 24.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=5a2e435683ee8925f800592738720a29c5a772ff5e85ff007051fc86515b02

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
from huggingface_hub import notebook_login
notebook_login()

In [34]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your

In [5]:
from datasets import load_dataset
dataset = load_dataset("anujsahani01/English-Marathi")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['english', 'marathi'],
        num_rows: 2637962
    })
    test: Dataset({
        features: ['english', 'marathi'],
        num_rows: 879321
    })
})

In [7]:
import fasttext
from huggingface_hub import hf_hub_download

In [8]:
model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
detect = fasttext.load_model(model_path)

In [9]:
def filter(data):
  data = data.replace('*', '').replace(':', '').replace(';', '').replace('$', '').replace('#', '').replace('@', '').replace('%', '').replace('&', '').replace('-', '').replace('_', '').replace('+', '').replace('~', '').replace('`', '').replace('>', '').replace('<', '').replace("|", '').replace('^', '').replace('(', '').replace(')', '').replace('{', '').replace('}', '').replace('!', '').replace('"', '').replace('[', '').replace(']', '').replace('删', '').replace('=', '').replace("'", '').replace('/', '').replace("⚡", " ").replace(".", '').replace('🐰', '').replace("【", '').replace('?', '')
  return data

In [10]:
def detect_lang(data):
  lang = detect.predict(data)[0][0]
  if lang == '__label__eng_Latn':
    return 'en'
  elif lang == '__label__mar_Deva':
    return 'mr'
  else:
    return None

In [11]:
import pandas as pd

In [12]:
def preprocessing(data):
  df = pd.DataFrame()
  df['final_english'] = data['english']
  df['final_marathi'] = data['marathi']
  l1 = len(df)
  df = df.sample(frac = 1)
  df.dropna(inplace = True)
  df.drop_duplicates(inplace = True)
  df['en_lang'] = list(map(lambda x: detect_lang(x), df['final_english']))
  df['mr_lang'] = list(map(lambda x: detect_lang(x), df['final_marathi']))
  df.drop(df[df['en_lang'] != 'en'].index , inplace = True)
  df.drop(df[df['mr_lang'] != 'mr'].index , inplace = True)
  df.reset_index(inplace = True)
  df['final_english'] = list(map(lambda x: filter(x) , df['final_english']))
  df['en_len'] = list(map(lambda x : len(x.split(' ')), df['final_english']))
  cols = list(df.columns)
  cols.remove('final_english')
  cols.remove('final_marathi')
  cols.remove('en_len')
  df.drop(cols, axis = 1, inplace = True)
  return df

In [13]:
df_train  = preprocessing(dataset['train'])

In [14]:
df_test = preprocessing(dataset['test'])

In [15]:
df_train.head(4)

,final_english,final_marathi,en_len
0,"Besides Aamir Khan, the film also stars Amitab...",दिवाळीच्या मुहुर्तावर प्रेक्षकांच्या भेटीला ये...,19
1,But he has anger towards me,"""मात्र त्यांच्या मनात माझ्या बद्दल राग आहे"""".""",6
2,"Meanwhile, Operation Shakti was also launched ...","दुसरीकडे, उत्तर प्रदेशची राजधानी असलेल्या लखनौ...",16
3,They live in another country,त्या दुसर्‍या देशात राहतात.,5


In [16]:
df_test.head(4)

,final_english,final_marathi,en_len
0,We both live in Boston,आम्ही दोघीही बॉस्टनमध्ये राहतो.,5
1,He is survived by a wife and two children,त्यांच्या पश्चात त्यांची पत्नी मंदाकिनी व दोन ...,9
2,This is dangerous for federalism and democracy,ते लोकशाहीला आणि देशासाठी घातक आहे.,7
3,"Which was with the deputy of the country, Serg...",बर्येशू नेहमी सिर्ग्य पौल याच्या निकट राहण्याच...,27


In [17]:
def split_len(df):
  range1 = []
  range2 = []
  range3 = []
  range4 = []
  for i in range(len(df)):
    if int(df['en_len'][i]) > 70:
      range4.append((df['final_english'][i], df['final_marathi'][i]))
    elif int(df['en_len'][i]) <= 70 and int(df['en_len'][i]) > 50:
      range3.append((df['final_english'][i], df['final_marathi'][i]))
    elif int(df['en_len'][i]) <= 50 and int(df['en_len'][i]) > 30:
      range2.append((df['final_english'][i], df['final_marathi'][i]))
    elif int(df['en_len'][i]) <= 30 and int(df['en_len'][i]) > 9:
      range1.append((df['final_english'][i], df['final_marathi'][i]))
  return range1, range2, range3, range4

In [18]:
def dataset_formation(range1, range2, range3, range4):
  df1 = pd.DataFrame()
  en_arr = []
  mr_arr = []
  l = len(range1)%4
  for i in range(0, len(range1) - l, 4):
    en, mr  = '', ''
    for j in range(i , i+4):
      en = en + range1[j][0] + '\n'
      mr = mr + range1[j][1] + '\n'
    en_arr.append(en)
    mr_arr.append(mr)
  df1['english'] = en_arr
  df1['marathi'] = mr_arr
  df2 = pd.DataFrame()
  en_arr = []
  mr_arr = []
  l = len(range2)%3
  for i in range(0, len(range2)-l, 3):
    en, mr  = '', ''
    for j in range(i , i+3):
      en = en + range2[j][0] + '\n'
      mr = mr + range2[j][1] + '\n'
    en_arr.append(en)
    mr_arr.append(mr)
  df2['english'] = en_arr
  df2['marathi'] = mr_arr
  df3 = pd.DataFrame()
  en_arr = []
  mr_arr = []
  l = len(range3)%2
  for i in range(0, len(range3)-l , 2):
    en, mr  = '', ''
    for j in range(i , i+2):
      en = en + range3[j][0] + '\n'
      mr = mr + range3[j][1] + '\n'
    en_arr.append(en)
    mr_arr.append(mr)
  df3['english'] = en_arr
  df3['marathi'] = mr_arr
  df4 = pd.DataFrame()
  en_arr, mr_arr = [], []
  for i in range(len(range4)):
    en_arr.append(range4[i][0])
    mr_arr.append(range4[i][1])
  df4['english'] = en_arr
  df4['marathi'] = mr_arr
  return df1, df2, df3, df4

In [19]:
range1, range2, range3, range4 = split_len(df_train)
df1, df2, df3, df4 = dataset_formation(range1, range2, range3, range4)
frames = [df1, df2, df3, df4]
df_final_train = pd.concat(frames)

In [20]:
range1, range2, range3, range4 = split_len(df_test)
df1, df2, df3, df4 = dataset_formation(range1, range2, range3, range4)
frames = [df1, df2, df3, df4]
df_final_test = pd.concat(frames)

In [21]:
print(len(df_final_train))
print(len(df_final_test))

283157
105456


In [22]:
frames = [df_final_train, df_final_test]
df_final = pd.concat(frames)

In [23]:
split = int(0.8*len(df_final))
df_final_train = df_final[:split]
df_final_test = df_final[split:]

In [24]:
df_final_train.reset_index(inplace = True)
df_final_train.drop(['index'] , axis = 1, inplace = True);

<ipython-input-24-d11541201666>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_train.drop(['index'] , axis = 1, inplace = True);


In [25]:
df_final_test.reset_index(inplace = True)
df_final_test.drop(['index'] , axis = 1, inplace = True);

<ipython-input-25-c969ad82d42a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_test.drop(['index'] , axis = 1, inplace = True);


In [26]:
en_len = list(map(lambda x : len(x.split(' ')) , df_final_train['english']))

In [27]:
import statistics
import numpy as np
print('Minimum:', min(en_len), 'Maximum: ' , max(en_len), 'Mean: ', np.mean(en_len), 'Mode:' , statistics.mode(en_len))
print(en_len.count(80))

Minimum: 37 Maximum:  10263 Mean:  66.22674901090417 Mode: 58
2343


In [28]:
df_final_train.head(4)

,english,marathi
0,"Besides Aamir Khan, the film also stars Amitab...",दिवाळीच्या मुहुर्तावर प्रेक्षकांच्या भेटीला ये...
1,She will be seen opposite Ranbir Kapoor in thi...,या चित्रपटात आपल्याला एक वेगळाच रणबीर कपूर पाह...
2,"Today, on the occasion of Consecration ceremon...",आज माता अन्नपूर्णा धाम इथे प्राणप्रतिष्ठा समयी...
3,A look at how dentists first learned to reliev...,"तरीसुद्धा, बऱ्‍याच लोकांना दंतवैद्याकडे जायला..."


In [29]:
df_final_test.head(4)

,english,marathi
0,Pakistan claims he was arrested in Balochistan...,कुलभूषणची अटक बलुचिस्तानातील माश्केल भागात केल...
1,He said the Union Government is working to end...,हा संघर्ष संपविण्यासाठी केंद्र सरकार प्रयत्नशी...
2,"Brothers and sisters, There is another good ne...","बंधू आणि भगिनींनो, पशु पालकांसाठी आणखी एक चांग..."
3,Watch Video BJP MLA greeted with garland of sh...,VIDEO: बैठकीतच भाजप खासदाराने आमदाराला चप्पलने...


In [30]:
from datasets import DatasetDict, Dataset

dataset = DatasetDict({
    'train' :  Dataset.from_pandas(df_final_train),
    'test' : Dataset.from_pandas(df_final_test)
})

In [31]:
dataset

DatasetDict({
    train: Dataset({
        features: ['english', 'marathi'],
        num_rows: 310890
    })
    test: Dataset({
        features: ['english', 'marathi'],
        num_rows: 77723
    })
})

In [35]:
from datasets import load_dataset

dataset.push_to_hub("anujsahani01/processed_en_mr")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/311 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/78 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]